In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from tensorflow.keras.preprocessing import image
import os

In [30]:
base_dir = 'D:\pest' 
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


In [31]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of training data for validation
)

In [32]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [33]:
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 1200 images belonging to 4 classes.


In [34]:
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 200 images belonging to 4 classes.


In [35]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [36]:
for layer in base_model.layers:
    layer.trainable = False


In [37]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.4)(x)  # Regularization
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.4)(x)
predictions = Dense(4, activation='softmax')(x)

In [38]:
model = Model(inputs=base_model.input, outputs=predictions)

In [39]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Lower initial learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [40]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_pest_detection_model.keras', monitor='val_accuracy', save_best_only=True),
    LearningRateScheduler(lr_schedule)
]

In [41]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=callbacks
)

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.4422 - loss: 11.6365 - val_accuracy: 0.9000 - val_loss: 9.0847 - learning_rate: 5.0000e-04
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.7621 - loss: 8.8400 - val_accuracy: 0.9200 - val_loss: 7.1934 - learning_rate: 5.0000e-04
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.8457 - loss: 7.0348 - val_accuracy: 0.9450 - val_loss: 5.8194 - learning_rate: 5.0000e-04
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.8924 - loss: 5.6818 - val_accuracy: 0.9550 - val_loss: 4.8034 - learning_rate: 5.0000e-04
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.9297 - loss: 4.6751 - val_accuracy: 0.9400 - val_loss: 4.0640 - learning_rate: 5.0000e-04
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.9442 - loss: 3.9003 - val_accuracy: 0.9850 - val_loss: 3.3515 - learning_rate: 5.0000e-04
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.9597 - loss

In [42]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 456ms/step - accuracy: 0.9835 - loss: 1.2962
Test Accuracy: 0.99


In [48]:
model.save('best_pest_detection_model.keras')


In [49]:
pest_info = {
    'aphids': {
        'lifecycle': "35-45 days after sowing",
        'control': ["Imidacloprid", "Acephate"]
    },
    'mites': {
        'lifecycle': "45-55 days after sowing",
        'control': ["Fipronil", "Diafenthiuron", "Beta-cyfluthrin + Imidacloprid"]
    },
    'bollworm': {
        'lifecycle': "80-90 days after sowing",
        'control': ["Profenofos + Cypermethrin", "Chlorantraniliprole + Lambda-cyhalothrin"]
    },
    'armyworm': {
        'lifecycle': "50-60 days after sowing",
        'control': ["Emamectin Benzoate 5 SG @ 0.4 g/l", "Chlorantraniliprole"]
    }
}

In [50]:
from tensorflow.keras.models import load_model

model = load_model('best_pest_detection_model.keras')

In [51]:
def predict_pest(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    # Make predictions
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions, axis=1)[0]
    class_names = ['aphids', 'mites', 'bollworm', 'armyworm']
    predicted_pest = class_names[class_idx]

    # Fetch pest-specific information
    lifecycle = pest_info[predicted_pest]['lifecycle']
    control = pest_info[predicted_pest]['control']

    # Display results
    print(f"Pest Detected: {predicted_pest.capitalize()}")
    print(f"Lifecycle Stage: {lifecycle}")
    print(f"Control Measures: {', '.join(control)}")

In [52]:

predict_pest(r'C:\Users\NN\Pictures\Screenshots\Screenshot 2024-11-15 023123.png')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 877ms/step
Pest Detected: Aphids
Lifecycle Stage: 35-45 days after sowing
Control Measures: Imidacloprid, Acephate
